In [1]:
from datasets import load_dataset

dataset = load_dataset("dim/verbalist_prompts")

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_names = list(dataset.column_names.keys())

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

In [11]:
temp = dataset.map(
    lambda x: {
        **x,
        "tokens_amount": len(
            tokenizer.encode(
                "\n".join(
                    x["conversation_text"],
                )
            )
        ),
    },
    num_proc=32,
)

Map (num_proc=32): 100%|██████████| 7188/7188 [00:00<00:00, 11271.64 examples/s]
num_proc must be <= 27. Reducing num_proc to 27 for dataset of size 27.
Map (num_proc=32): 100%|██████████| 64006/64006 [00:05<00:00, 11082.95 examples/s]


In [10]:
len(tokenizer.encode("Hello world"))

3

In [20]:
temp[dataset_names[0]].to_pandas().describe().transpose()

,count,mean,std,min,25%,50%,75%,max
tokens_amount,2289.0,468.678899,295.086439,17.0,264.0,410.0,618.0,2332.0


In [22]:
temp[dataset_names[0]].to_pandas().describe().transpose().columns.tolist()

['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']

In [29]:
temp[dataset_names[0]].to_pandas().describe().transpose().iloc[0].tolist()

[2289.0,
 468.6788990825688,
 295.0864391190051,
 17.0,
 264.0,
 410.0,
 618.0,
 2332.0]

In [34]:
import pandas as pd

column_names = temp[dataset_names[0]].to_pandas().describe().transpose().columns.tolist()

length_statistics = []

for dataset_name in dataset_names:
    dataset_stats = temp[dataset_name].to_pandas().describe().transpose()
    values = dataset_stats.iloc[0].tolist()
    dict_stats = {}
    dict_stats['dataset_name'] = dataset_name
    for column_name, value in zip(column_names, values):
        dict_stats[column_name] = value
    length_statistics.append(dict_stats)
    
pd.DataFrame(length_statistics).to_csv("./verbalist/datasets/datasets_statistics/verbalist_datasets_statistics.csv", index=False)